In [ ]:
import geopandas as gpd
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import modin.pandas as pd
import numpy as np
import shapely
from modin.config import ProgressBar
from tqdm.auto import tqdm

ProgressBar.enable()

# Parameter space exploration

In [ ]:
wforms = (
    pd.read_parquet("wforms-bat.parquet")
    .join(pd.read_parquet("wforms-ann.parquet"), how="left")
    .join(pd.read_parquet("wforms-occ.parquet", columns=["rho", "cvx"]), how="left")
)

wforms.loc[(wforms["fst_batch"] | wforms["snd_batch"]) & wforms["tt_att"], "status"] = -1
wforms.loc[(wforms["fst_batch"] | wforms["snd_batch"]) & wforms.index.str.startswith("#"), "status"] = np.nan

wforms

In [ ]:
rho_thresholdsp = pd.Series(np.linspace(0.2, 1, 50))
rho_thresholdsn = pd.Series(np.linspace(-1, -0.2, 50))

testp = wforms[(~wforms["tt_att"]) & wforms["fst_batch"] & (wforms["rho"] > 0.2)]
testn = wforms[(~wforms["tt_att"]) & wforms["fst_batch"] & (wforms["rho"] < -0.2)]


fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(
    rho_thresholdsp,
    rho_thresholdsp.apply(lambda t: (testp["rho"] >= t).sum()),
    color="red"
)

ax2.plot(
    rho_thresholdsp,
    rho_thresholdsp.apply(lambda t: ((testp["rho"] >= t) & testp["status"].ge(0)).mean())/rho_thresholdsp.apply(lambda t: (testp["rho"] >= t).sum()),
)


ax1.plot(
    rho_thresholdsn,
    rho_thresholdsn.apply(lambda t: (testn["rho"] <= t).sum()),
    color="red"
)

ax2.plot(
    rho_thresholdsn,
    rho_thresholdsn.apply(lambda t: ((testn["rho"] <= t) & testn["status"].ge(0)).mean())/rho_thresholdsn.apply(lambda t: (testn["rho"] <= t).sum()),
)


fig.tight_layout()
plt.show()

In [ ]:
fst_thresholds = pd.Series(np.arange(7,366,5))
lst_thresholds = pd.Series(np.arange(351,366))

testf = wforms[wforms["snd_batch"] & (wforms["fst"] > 7)]
testl = wforms[wforms["snd_batch"] & (wforms["lst"] > 351)]


fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(
    fst_thresholds,
    fst_thresholds.apply(lambda t: (testf["fst"] >= t).sum()),
    color="red"
)

ax2.plot(
    fst_thresholds,
    fst_thresholds.apply(lambda t: ((testf["fst"] >= t) & testf["status"].eq(1)).mean()),#/fst_thresholds.apply(lambda t: (testf["fst"] >= t).sum()),
)


ax1.plot(
    lst_thresholds,
    lst_thresholds.apply(lambda t: (testl["lst"] >= t).sum()),
    color="red"
)

ax2.plot(
    lst_thresholds,
    lst_thresholds.apply(lambda t: ((testl["lst"] >= t) & testl["status"].eq(1)).mean()),#/lst_thresholds.apply(lambda t: (testl["lst"] >= t).sum()),
)


fig.tight_layout()
plt.show()

# convexity

In [ ]:
thresholds = pd.Series(np.linspace(-1,+1,20))

test = wforms[wforms["snd_batch"] & ~wforms["cvx"].isna()]

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(
    thresholds,
    thresholds.apply(lambda t: (test["cvx"] <= t).sum()),
    color="red",
)

ax2.plot(
    thresholds,
    thresholds.apply(lambda t: ((test["cvx"] <= t) & test["status"].eq(1)).mean()),
    color="blue",
    label="yield",
)

plt.legend()

fig.tight_layout()
plt.show()

In [ ]:
thresholds = pd.Series(np.linspace(-1,+1,20))

test = wforms[wforms["snd_batch"] & ~wforms["cvx"].isna()]

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(
    thresholds,
    thresholds.apply(lambda t: (test["cvx"] >= t).sum()),
    color="red",
)

ax2.plot(
    thresholds,
    thresholds.apply(lambda t: ((test["cvx"] >= t) & test["status"].eq(1)).mean()),
    color="blue",
    label="yield",
)

plt.legend()

fig.tight_layout()
plt.show()

In [ ]:
thresholds = pd.Series(np.linspace(0,+1,100))

test = wforms[wforms["snd_batch"] & ~wforms["cvx"].isna()]

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(
    thresholds,
    thresholds.apply(lambda t: ((test["cvx"]) >= t).sum()),
    color="red",
)

ax2.plot(
    thresholds,
    thresholds.apply(lambda t: (((test["cvx"]) >= t) & test["status"].eq(1)).mean()),#/thresholds.apply(lambda t: ((test["cvx"]) >= t).sum()),
    color="blue",
    label="yield",
)

plt.legend()

fig.tight_layout()
plt.show()

In [ ]:
thresholds = pd.Series(np.linspace(-1,0,100))

test = wforms[wforms["snd_batch"] & ~wforms["cvx"].isna()]

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(
    thresholds,
    thresholds.apply(lambda t: ((test["cvx"]) <= t).sum()),
    color="red",
)

ax2.plot(
    thresholds,
    thresholds.apply(lambda t: (((test["cvx"]) <= t) & test["status"].eq(1)).mean()),#/thresholds.apply(lambda t: ((test["cvx"]) <= t).sum()),
    color="blue",
    label="yield",
)

plt.legend()

fig.tight_layout()
plt.show()

In [ ]:
thresholds = pd.Series(np.linspace(0,+1,100))

test = wforms[wforms["snd_batch"] & ~wforms["cvx"].isna()]

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(
    thresholds,
    thresholds.apply(lambda t: (abs(test["cvx"]) >= t).sum()),
    color="red",
)

ax2.plot(
    thresholds,
    thresholds.apply(lambda t: ((abs(test["cvx"]) >= t) & test["status"].ge(0)).mean())/thresholds.apply(lambda t: (abs(test["cvx"]) >= t).sum()),
    color="blue",
    label="yield",
)

plt.legend()

fig.tight_layout()
plt.show()